In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split
import librosa
import audioread
import os


In [24]:
def extract_features(file_path, sr=22050, n_mfcc=40):
    try:
        audio, sample_rate = librosa.load(file_path, sr=sr)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
        return np.mean(mfccs.T, axis=0)
    except audioread.NoBackendError:
        print(f"Cannot process file: {file_path}")
        return None  # Handle or log invalid files gracefully


In [25]:
# Dataset preparation
audio_files = []  # List of file paths
labels = []       # List of class indices

data_dir = "data_dir/"
class_mapping = {"door_bell": 0, "car_horn": 1}  # Example

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        for file_name in os.listdir(label_dir):
            file_path = os.path.join(label_dir, file_name)
            audio_files.append(file_path)
            labels.append(class_mapping[label])  # Convert class name to index


In [26]:
X = np.array([extract_features(f) for f in audio_files])
y = np.array(labels)

In [27]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
num_classes = len(class_mapping)
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

# Build a simple neural network
model = Sequential([
    Dense(256, input_shape=(X_train.shape[1],), activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Multi-class classification
    metrics=['accuracy']
)


c:\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.fit(
    X_train, y_train_one_hot,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test_one_hot)
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5625 - loss: 9.9972 - val_accuracy: 0.6000 - val_loss: 1.8252
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.6875 - loss: 6.2516 - val_accuracy: 0.6000 - val_loss: 5.9015
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.6875 - loss: 11.9891 - val_accuracy: 0.6000 - val_loss: 6.2965
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.5625 - loss: 9.5403 - val_accuracy: 0.6000 - val_loss: 5.8111
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.6250 - loss: 10.8997 - val_accuracy: 0.6000 - val_loss: 1.2744
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.6250 - loss: 13.8434 - val_accuracy: 0.8000 - val_loss: 0.8547
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.5625 - loss: 18.7367 - val_accuracy: 0.8000 - val_loss: 1.5750
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.6875 - loss: 9.6435 - val_accuracy: 0.8000 - val_loss: 1.74

In [30]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_one_hot, verbose=2)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


1/1 - 0s - 45ms/step - accuracy: 1.0000 - loss: 2.4319e-06
Test Loss: 0.0000, Test Accuracy: 1.0000


In [32]:
model.save('sensear_prototype.keras')

In [34]:

# Load the trained model (replace 'model_path' with the path to your model)
model = tf.keras.models.load_model("sensear_prototype.keras")

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open("sensear_prototype.tflite", "wb") as f:
    f.write(tflite_model)

print("Model has been successfully converted and saved as model.tflite.")


c:\Python310\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


INFO:tensorflow:Assets written to: C:\Users\MUKHAM~1\AppData\Local\Temp\tmp9w182hxh\assets


INFO:tensorflow:Assets written to: C:\Users\MUKHAM~1\AppData\Local\Temp\tmp9w182hxh\assets


Saved artifact at 'C:\Users\MUKHAM~1\AppData\Local\Temp\tmp9w182hxh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 40), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  2024131848224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2024131653904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2024132708656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2024127682992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2024133920016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2024133921072: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model has been successfully converted and saved as model.tflite.
